In [3]:
# Imports and data loading
import numpy as np
import cv2 as cv
import os
import math
from PIL import Image

import colour
import imageio
imageio.plugins.freeimage.download()

IMAGE_DTYPE='float16'

NUM_IMAGES=32
IMG_NAME="chromeball."
INPUT_PATH="../../data/simulated"
OUTPUT_PATH="../../data/output"

# Load images
frames=[]
for i in range(2, NUM_IMAGES+1):
    file = os.path.join(os.path.abspath(INPUT_PATH), f"{IMG_NAME}{i:04}.exr")
    frames.append(colour.read_image(file, bit_depth=IMAGE_DTYPE, method='Imageio'))
original = colour.read_image(os.path.join(os.path.abspath(INPUT_PATH), f"{IMG_NAME}0001.exr"), bit_depth=IMAGE_DTYPE, method='Imageio')
original = original[:,:,:3] # Only use red channel

Imageio: 'libfreeimage-3.16.0-linux64.so' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/freeimage/libfreeimage-3.16.0-linux64.so (4.6 MB)
Downloading: 8192/4830080 bytes (0.2%4830080/4830080 bytes (100.0%)
  Done
File saved as /home/raumfisch/.imageio/freeimage/libfreeimage-3.16.0-linux64.so.


In [3]:
# Find center and radius of chromeball (use alpha mask here)
mask = (frames[0][...,3] * 255).astype('uint8')
center = (0,0)
radius = 0

if True:
    gray = mask
    #cv.medianBlur(mask, 5)

    #cv.Smooth(orig, orig, cv.CV_GAUSSIAN, 5, 5)
    #cv.CvtColor(orig, grey_scale, cv.CV_RGB2GRAY)
    #cv.Erode(grey_scale, processed, None, 10)
    #cv.Dilate(processed, processed, None, 10)
    #cv.Canny(processed, processed, 5, 70, 3)
    #cv.Smooth(processed, processed, cv.CV_GAUSSIAN, 15, 15)
    
    rows = gray.shape[0]
    circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, rows / 8,
                                param1=100, param2=30,
                                minRadius=100, maxRadius=1080)
        
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = np.array((i[0], i[1]))
            radius = i[2]
            
            #orig = np.dstack((gray, gray, gray))
            # circle center
            #cv.circle(orig, center, 1, (0, 100, 100), 3)
            # circle outline
            #cv.circle(orig, center, radius, (255, 0, 255), 3)



In [29]:
# Find reflections in each image
i = 0
orig = (np.clip(original, 0, 1) * 255).astype('uint8')

for frame in frames:
    frame = frame[:,:,:1] # Only use red channel
    frame = (np.clip(frame, 0, 1) * 255).astype('uint8')
    rows = frame.shape[0]
    circles = cv.HoughCircles(frame, cv.HOUGH_GRADIENT, 1, rows / 2,
                                param1=100, param2=5,
                                minRadius=1, maxRadius=100)
        
    if circles is not None:
        for c in circles[0, :]:
            point = (int(c[0]+0.5),int(c[1]+0.5))
            cv.line(orig, point, center, (0,255,0))

            # Calculate spherical coordinates (O = center, I = reflection)
            refl_c = np.array((c[0], c[1]))
            OI = refl_c - center
            length = np.linalg.norm(OI) / radius
            phi = np.angle(-OI[1]+OI[0]*1j, deg=True)
            print(f"{i}: {phi} - {length}")
            # Calculate angles relative to z axis
            z = math.sin(length*math.pi/2)
            print(f">: {z} - {length}")

    i += 1

if True:
    file = os.path.join(OUTPUT_PATH, f"positions.png")
    colour.write_image(orig, file, 'uint8', method='Imageio')

0: -89.70002483769609 - 0.2776201069355011
1: -89.8438810961417 - 0.5334322452545166
2: -89.88654315916008 - 0.734013020992279
3: -89.9052963184549 - 0.8793616890907288
4: -89.37627587340576 - 0.9346483945846558
7: 89.90832683100679 - 0.9084314107894897
8: 89.88787532999517 - 0.7427339553833008
9: 89.84040214115332 - 0.5218043923377991
10: 89.69360837772929 - 0.2718062102794647
11: -45.0 - 0.0020555430091917515
12: -0.35150343987335203 - 0.23692306876182556
13: -53.810732985984906 - 0.41601771116256714
14: -64.46866362968093 - 0.6845821738243103
15: -61.33819347073664 - 0.9060841798782349
16: -35.7633394038445 - 0.8580088019371033
17: 35.70902914508374 - 0.903964638710022
18: 60.99745141008237 - 0.9323218464851379
19: 64.36339781718272 - 0.6819602847099304
20: 53.57299836361139 - 0.41367512941360474
21: -0.1818907524971836 - 0.4578511416912079
22: -35.66243704937296 - 0.6207773685455322
23: -36.29152612504515 - 0.950339138507843
24: -1.1546423253669917 - 0.9376903772354126
25: 35.44432

In [5]:
# Save images
i = 1
for frame in frames:
    file = os.path.join(OUTPUT_PATH, f"{IMG_NAME}{i:04}.exr")
    colour.write_image(frame, file, 'float32', method='Imageio')
    i += 1